In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import tensorflow as tf
import numpy as np
from rotary_embedding_tensorflow import apply_rotary_emb, RotaryEmbedding
from fast_transformer import FastTransformer

In [3]:
from malaya.text.bpe import WordPieceTokenizer

In [4]:
tokenizer = WordPieceTokenizer('BERT.wordpiece', do_lower_case = False)
# tokenizer.tokenize('halo nama sayacomel')

In [5]:
import pickle

with open('entities-fastformer.pkl', 'rb') as fopen:
    train_X, train_Y, test_X, test_Y = pickle.load(fopen)

In [6]:
epoch = 2
batch_size = 32
warmup_proportion = 0.1
num_train_steps = int(len(train_X) / batch_size * epoch)
num_warmup_steps = int(num_train_steps * warmup_proportion)

In [7]:
import optimization

In [8]:
def create_initializer(initializer_range=0.02):
    return tf.truncated_normal_initializer(stddev=initializer_range)

class Model:
    def __init__(
        self,
        dimension_output,
        learning_rate = 2e-5,
        training = True,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        mask = tf.math.not_equal(self.X, 0)
        mask = tf.cast(mask, tf.bool)
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.maxlen = tf.shape(self.X)[1]
        self.lengths = tf.count_nonzero(self.X, 1)
        
        self.model = FastTransformer(
            num_tokens = 32000,
            dim = 768,
            depth = 12,
            heads = 12,
            max_seq_len = 2048,
            absolute_pos_emb = True,
            mask = mask
        )
        self.logits = self.model(self.X)[0]
        logits = tf.layers.dense(self.logits, dimension_output,
                                         kernel_initializer=create_initializer())
        
        y_t = self.Y
        log_likelihood, transition_params = tf.contrib.crf.crf_log_likelihood(
            logits, y_t, self.lengths
        )
        self.cost = tf.reduce_mean(-log_likelihood)
        self.optimizer = tf.train.AdamOptimizer(
            learning_rate = learning_rate
        ).minimize(self.cost)
        mask = tf.sequence_mask(self.lengths, maxlen = self.maxlen)
        self.tags_seq, tags_score = tf.contrib.crf.crf_decode(
            logits, transition_params, self.lengths
        )
        self.tags_seq = tf.identity(self.tags_seq, name = 'logits')

        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(self.tags_seq, mask)
        mask_label = tf.boolean_mask(y_t, mask)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [9]:
import json

with open('dictionary-entities.json') as fopen:
    dictionary = json.load(fopen)

tag2idx = dictionary['tag2idx']
idx2tag = {int(k): v for k, v in dictionary['idx2tag'].items()}

In [10]:
dimension_output = len(tag2idx)
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)

sess.run(tf.global_variables_initializer())
var_lists = tf.trainable_variables()

Instructions for updating:
reduction_indices is deprecated, use axis instead
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Tensor("fast_transformer/pre_norm/fast_attention/Select:0", shape=(?, 12, ?), dtype=float32)
Tensor("fast_transformer/pre_norm/fast_attention/Select:0", shape=(?, 12, ?), dtype=float32)
Tensor("fast_transformer/pre_norm_2/fast_attention_1/Select:0", shape=(?, 12, ?), dtype=float32)
Tensor("fast_transformer/pre_norm_2/fast_attention_1/Select:0", shape=(?, 12, ?), dtype=float32)
Tensor("fast_transformer/pre_norm_4/fast_attention_2/Select:0", shape=(?, 12, ?), dtype=float32)
Tensor("fast_transformer/pre_norm_4/fast_attention_2/Select:0", shape=(?, 12, ?), dtype=float32)
Tensor("fast_transformer/pre_norm_6/fast_attention_3/Sel

In [11]:
import collections
import re

def get_assignment_map_from_checkpoint(tvars, init_checkpoint):
    """Compute the union of the current variables and checkpoint variables."""
    assignment_map = {}
    initialized_variable_names = {}

    name_to_variable = collections.OrderedDict()
    for var in tvars:
        name = var.name
        m = re.match('^(.*):\\d+$', name)
        if m is not None:
            name = m.group(1)
        name_to_variable[name] = var

    init_vars = tf.train.list_variables(init_checkpoint)

    assignment_map = collections.OrderedDict()
    for x in init_vars:
        (name, var) = (x[0], x[1])
        if name not in name_to_variable:
            continue
        assignment_map[name] = name_to_variable[name]
        initialized_variable_names[name] = 1
        initialized_variable_names[name + ':0'] = 1

    return (assignment_map, initialized_variable_names)

In [12]:
tvars = tf.trainable_variables()
checkpoint = 'fastformer-base/model.ckpt-500000'
assignment_map, initialized_variable_names = get_assignment_map_from_checkpoint(tvars, 
                                                                                checkpoint)

In [13]:
saver = tf.train.Saver(var_list = assignment_map)
saver.restore(sess, checkpoint)

INFO:tensorflow:Restoring parameters from fastformer-base/model.ckpt-500000


In [14]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [15]:
string = 'KUALA LUMPUR: Sempena sambutan Aidilfitri minggu depan, Perdana Menteri Tun Dr Mahathir Mohamad dan Menteri Pengangkutan Anthony Loke Siew Fook menitipkan pesanan khas kepada orang ramai yang mahu pulang ke kampung halaman masing-masing. Dalam video pendek terbitan Jabatan Keselamatan Jalan Raya (JKJR) itu, Dr Mahathir menasihati mereka supaya berhenti berehat dan tidur sebentar  sekiranya mengantuk ketika memandu.'

import re

def entities_textcleaning(string, lowering = False):
    """
    use by entities recognition, pos recognition and dependency parsing
    """
    string = re.sub('[^A-Za-z0-9\-\/() ]+', ' ', string)
    string = re.sub(r'[ ]+', ' ', string).strip()
    original_string = string.split()
    if lowering:
        string = string.lower()
    string = [
        (original_string[no], word.title() if word.isupper() else word)
        for no, word in enumerate(string.split())
        if len(word)
    ]
    return [s[0] for s in string], [s[1] for s in string]

def parse_X(left):
    bert_tokens = ['[CLS]']
    for no, orig_token in enumerate(left):
        t = tokenizer.tokenize(orig_token)
        bert_tokens.extend(t)
    bert_tokens.append("[SEP]")
    input_mask = [1] * len(bert_tokens)
    return tokenizer.convert_tokens_to_ids(bert_tokens), bert_tokens, input_mask

sequence = entities_textcleaning(string)[1]
parsed_sequence, bert_sequence, input_mask = parse_X(sequence)

In [16]:
predicted = sess.run(model.tags_seq,
                feed_dict = {
                    model.X: [parsed_sequence],
                },
        )[0]

In [17]:
def merge_wordpiece_tokens_tagging(x, y):
    new_paired_tokens = []
    n_tokens = len(x)
    rejected = ['[CLS]', '[SEP]', '[PAD]']

    i = 0

    while i < n_tokens:
        current_token, current_label = x[i], y[i]
        if current_token.startswith('##'):
            previous_token, previous_label = new_paired_tokens.pop()
            merged_token = previous_token
            merged_label = [previous_label]
            while current_token.startswith('##'):
                merged_token = merged_token + current_token.replace('##', '')
                merged_label.append(current_label)
                i = i + 1
                current_token, current_label = x[i], y[i]
            merged_label = merged_label[0]
            new_paired_tokens.append((merged_token, merged_label))

        else:
            new_paired_tokens.append((current_token, current_label))
            i = i + 1

    words = [
        i[0]
        for i in new_paired_tokens
        if i[0] not in rejected
    ]
    labels = [i[1] for i in new_paired_tokens if i[0] not in rejected]
    return words, labels

In [18]:
merged = merge_wordpiece_tokens_tagging(bert_sequence, [idx2tag[d] for d in predicted])
list(zip(merged[0], merged[1]))

[('Kuala', 'event'),
 ('Lumpur', 'X'),
 ('Sempena', 'X'),
 ('sambutan', 'organization'),
 ('Aidilfitri', 'X'),
 ('minggu', 'person'),
 ('depan', 'X'),
 ('Perdana', 'X'),
 ('Menteri', 'X'),
 ('Tun', 'organization'),
 ('Dr', 'person'),
 ('Mahathir', 'X'),
 ('Mohamad', 'location'),
 ('dan', 'X'),
 ('Menteri', 'X'),
 ('Pengangkutan', 'organization'),
 ('Anthony', 'X'),
 ('Loke', 'X'),
 ('Siew', 'location'),
 ('Fook', 'X'),
 ('menitipkan', 'X'),
 ('pesanan', 'organization'),
 ('khas', 'X'),
 ('kepada', 'person'),
 ('orang', 'X'),
 ('ramai', 'X'),
 ('yang', 'X'),
 ('mahu', 'organization'),
 ('pulang', 'X'),
 ('ke', 'X'),
 ('kampung', 'event'),
 ('halaman', 'X'),
 ('masing', 'event'),
 ('-', 'person'),
 ('masing', 'X'),
 ('Dalam', 'event'),
 ('video', 'person'),
 ('pendek', 'X'),
 ('terbitan', 'X'),
 ('Jabatan', 'X'),
 ('Keselamatan', 'event'),
 ('Jalan', 'organization'),
 ('Raya', 'X'),
 ('(', 'event'),
 ('Jkjr', 'X'),
 (')', 'event'),
 ('itu', 'X'),
 ('Dr', 'event'),
 ('Mahathir', 'person')

In [19]:
from tqdm import tqdm
import time

for e in range(1):
    lasttime = time.time()
    train_acc, train_loss, test_acc, test_loss = [], [], [], []
    pbar = tqdm(
        range(0, len(train_X), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_X))
        batch_x = train_X[i : index]
        batch_y = train_Y[i : index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_y = pad_sequences(batch_y, padding='post')
        
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.X: batch_x,
                model.Y: batch_y,
            },
        )
        assert not np.isnan(cost)
        train_loss.append(cost)
        train_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
    
    pbar = tqdm(
        range(0, len(test_X), batch_size), desc = 'test minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(test_X))
        batch_x = test_X[i : index]
        batch_y = test_Y[i : index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_y = pad_sequences(batch_y, padding='post')
        
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.X: batch_x,
                model.Y: batch_y,
            },
        )
        assert not np.isnan(cost)
        test_loss.append(cost)
        test_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
    
    train_loss = np.mean(train_loss)
    train_acc = np.mean(train_acc)
    test_loss = np.mean(test_loss)
    test_acc = np.mean(test_acc)

    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (e, train_loss, train_acc, test_loss, test_acc)
    )
    predicted = sess.run(model.tags_seq,
                feed_dict = {
                    model.X: [parsed_sequence],
                },
        )[0]
    merged = merge_wordpiece_tokens_tagging(bert_sequence, [idx2tag[d] for d in predicted])
    print(list(zip(merged[0], merged[1])))

test minibatch loop: 100%|██████████| 4688/4688 [17:29<00:00,  4.47it/s, accuracy=0.979, cost=5.69]


time taken: 14207.170645713806
epoch: 0, training loss: 1.841172, training acc: 0.991068, valid loss: 6.252518, valid acc: 0.980113

[('Kuala', 'location'), ('Lumpur', 'location'), ('Sempena', 'location'), ('sambutan', 'OTHER'), ('Aidilfitri', 'event'), ('minggu', 'time'), ('depan', 'time'), ('Perdana', 'person'), ('Menteri', 'person'), ('Tun', 'person'), ('Dr', 'person'), ('Mahathir', 'person'), ('Mohamad', 'person'), ('dan', 'OTHER'), ('Menteri', 'person'), ('Pengangkutan', 'person'), ('Anthony', 'person'), ('Loke', 'person'), ('Siew', 'person'), ('Fook', 'person'), ('menitipkan', 'X'), ('pesanan', 'OTHER'), ('khas', 'OTHER'), ('kepada', 'OTHER'), ('orang', 'OTHER'), ('ramai', 'OTHER'), ('yang', 'OTHER'), ('mahu', 'OTHER'), ('pulang', 'OTHER'), ('ke', 'OTHER'), ('kampung', 'OTHER'), ('halaman', 'OTHER'), ('masing', 'OTHER'), ('-', 'X'), ('masing', 'X'), ('Dalam', 'OTHER'), ('video', 'OTHER'), ('pendek', 'OTHER'), ('terbitan', 'OTHER'), ('Jabatan', 'organization'), ('Keselamatan', 'or

In [20]:
predicted = sess.run(model.tags_seq,
            feed_dict = {
                model.X: [parsed_sequence],
            },
    )[0]
merged = merge_wordpiece_tokens_tagging(bert_sequence, [idx2tag[d] for d in predicted])
print(list(zip(merged[0], merged[1])))

[('Kuala', 'location'), ('Lumpur', 'location'), ('Sempena', 'location'), ('sambutan', 'OTHER'), ('Aidilfitri', 'event'), ('minggu', 'time'), ('depan', 'time'), ('Perdana', 'person'), ('Menteri', 'person'), ('Tun', 'person'), ('Dr', 'person'), ('Mahathir', 'person'), ('Mohamad', 'person'), ('dan', 'OTHER'), ('Menteri', 'person'), ('Pengangkutan', 'person'), ('Anthony', 'person'), ('Loke', 'person'), ('Siew', 'person'), ('Fook', 'person'), ('menitipkan', 'X'), ('pesanan', 'OTHER'), ('khas', 'OTHER'), ('kepada', 'OTHER'), ('orang', 'OTHER'), ('ramai', 'OTHER'), ('yang', 'OTHER'), ('mahu', 'OTHER'), ('pulang', 'OTHER'), ('ke', 'OTHER'), ('kampung', 'OTHER'), ('halaman', 'OTHER'), ('masing', 'OTHER'), ('-', 'X'), ('masing', 'X'), ('Dalam', 'OTHER'), ('video', 'OTHER'), ('pendek', 'OTHER'), ('terbitan', 'OTHER'), ('Jabatan', 'organization'), ('Keselamatan', 'organization'), ('Jalan', 'location'), ('Raya', 'location'), ('(', 'location'), ('Jkjr', 'X'), (')', 'X'), ('itu', 'OTHER'), ('Dr', 'pe

In [21]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'fastformer-base-entities/model.ckpt')

'fastformer-base-entities/model.ckpt'

In [22]:
def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            out_i.append(idx2tag[p])
        out.append(out_i)
    return out

In [23]:
real_Y, predict_Y = [], []

pbar = tqdm(
    range(0, len(test_X), batch_size), desc = 'validation minibatch loop'
)
for i in pbar:
    index = min(i + batch_size, len(test_X))
    batch_x = test_X[i : index]
    batch_y = test_Y[i : index]
    batch_x = pad_sequences(batch_x, padding='post')
    batch_y = pad_sequences(batch_y, padding='post')
    predicted = pred2label(sess.run(model.tags_seq,
            feed_dict = {
                model.X: batch_x,
            },
    ))
    real = pred2label(batch_y)
    predict_Y.extend(predicted)
    real_Y.extend(real)

validation minibatch loop: 100%|██████████| 4688/4688 [18:12<00:00,  4.29it/s]


In [24]:
temp_real_Y = []
for r in real_Y:
    temp_real_Y.extend(r)
    
temp_predict_Y = []
for r in predict_Y:
    temp_predict_Y.extend(r)

In [25]:
len(temp_real_Y), len(temp_predict_Y)

(14464576, 14464576)

In [26]:
from sklearn.metrics import classification_report
print(classification_report(temp_real_Y, temp_predict_Y, digits = 5))

              precision    recall  f1-score   support

       OTHER    0.98377   0.98598   0.98487   4877783
         PAD    1.00000   1.00000   1.00000   3235654
           X    0.99427   0.98744   0.99084   3728922
       event    0.87362   0.89726   0.88528     38359
         law    0.95097   0.88134   0.91483     33011
    location    0.95822   0.98902   0.97338   1508352
organization    0.89477   0.87946   0.88705    231711
      person    0.97205   0.91806   0.94428    483093
    quantity    0.94198   0.92804   0.93495    133054
        time    0.93342   0.93521   0.93431    194637

    accuracy                        0.98414  14464576
   macro avg    0.95031   0.94018   0.94498  14464576
weighted avg    0.98420   0.98414   0.98411  14464576



In [27]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'logits' in n.name
        or 'alphas' in n.name
        or 'self/Softmax' in n.name)
        and 'adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
        and 'ReadVariableOp' not in n.name
        and 'AssignVariableOp' not in n.name
        and '/Assign' not in n.name
        and '/Adam' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'Placeholder_1',
 'dense/kernel',
 'dense/bias',
 'transitions',
 'logits']

In [28]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [29]:
freeze_graph('fastformer-base-entities', strings)

INFO:tensorflow:Restoring parameters from fastformer-base-entities/model.ckpt
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 163 variables.
INFO:tensorflow:Converted 163 variables to const ops.
7024 ops in the final graph.


In [30]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [31]:
g = load_graph('fastformer-base-entities/frozen_model.pb')
x = g.get_tensor_by_name('import/Placeholder:0')
logits = g.get_tensor_by_name('import/logits:0')
test_sess = tf.InteractiveSession(graph = g)

In [32]:
%%time

predicted = test_sess.run(logits,
            feed_dict = {
                x: [parsed_sequence],
            },
    )[0]
merged = merge_wordpiece_tokens_tagging(bert_sequence, [idx2tag[d] for d in predicted])
print(list(zip(merged[0], merged[1])))

[('Kuala', 'location'), ('Lumpur', 'location'), ('Sempena', 'location'), ('sambutan', 'OTHER'), ('Aidilfitri', 'event'), ('minggu', 'time'), ('depan', 'time'), ('Perdana', 'person'), ('Menteri', 'person'), ('Tun', 'person'), ('Dr', 'person'), ('Mahathir', 'person'), ('Mohamad', 'person'), ('dan', 'OTHER'), ('Menteri', 'person'), ('Pengangkutan', 'person'), ('Anthony', 'person'), ('Loke', 'person'), ('Siew', 'person'), ('Fook', 'person'), ('menitipkan', 'X'), ('pesanan', 'OTHER'), ('khas', 'OTHER'), ('kepada', 'OTHER'), ('orang', 'OTHER'), ('ramai', 'OTHER'), ('yang', 'OTHER'), ('mahu', 'OTHER'), ('pulang', 'OTHER'), ('ke', 'OTHER'), ('kampung', 'OTHER'), ('halaman', 'OTHER'), ('masing', 'OTHER'), ('-', 'X'), ('masing', 'X'), ('Dalam', 'OTHER'), ('video', 'OTHER'), ('pendek', 'OTHER'), ('terbitan', 'OTHER'), ('Jabatan', 'organization'), ('Keselamatan', 'organization'), ('Jalan', 'location'), ('Raya', 'location'), ('(', 'location'), ('Jkjr', 'X'), (')', 'X'), ('itu', 'OTHER'), ('Dr', 'pe

In [33]:
from tensorflow.tools.graph_transforms import TransformGraph

In [34]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

input_nodes = [
    'Placeholder',
]
output_nodes = [
    'logits',
]

pb = 'fastformer-base-entities/frozen_model.pb'

input_graph_def = tf.GraphDef()
with tf.gfile.FastGFile(pb, 'rb') as f:
    input_graph_def.ParseFromString(f.read())

transformed_graph_def = TransformGraph(input_graph_def, 
                                           input_nodes,
                                           output_nodes, transforms)
    
with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
    f.write(transformed_graph_def.SerializeToString())

Instructions for updating:
Use tf.gfile.GFile.


In [35]:
g = load_graph('fastformer-tiny-entities/frozen_model.pb.quantized')
x = g.get_tensor_by_name('import/Placeholder:0')
logits = g.get_tensor_by_name('import/logits:0')
test_sess = tf.InteractiveSession(graph = g)

In [36]:
%%time

predicted = test_sess.run(logits,
            feed_dict = {
                x: [parsed_sequence],
            },
    )[0]
merged = merge_wordpiece_tokens_tagging(bert_sequence, [idx2tag[d] for d in predicted])
print(list(zip(merged[0], merged[1])))

[('Kuala', 'location'), ('Lumpur', 'location'), ('Sempena', 'location'), ('sambutan', 'OTHER'), ('Aidilfitri', 'event'), ('minggu', 'time'), ('depan', 'time'), ('Perdana', 'person'), ('Menteri', 'person'), ('Tun', 'person'), ('Dr', 'person'), ('Mahathir', 'person'), ('Mohamad', 'person'), ('dan', 'OTHER'), ('Menteri', 'person'), ('Pengangkutan', 'person'), ('Anthony', 'person'), ('Loke', 'person'), ('Siew', 'event'), ('Fook', 'event'), ('menitipkan', 'person'), ('pesanan', 'OTHER'), ('khas', 'OTHER'), ('kepada', 'OTHER'), ('orang', 'OTHER'), ('ramai', 'OTHER'), ('yang', 'OTHER'), ('mahu', 'OTHER'), ('pulang', 'OTHER'), ('ke', 'OTHER'), ('kampung', 'OTHER'), ('halaman', 'OTHER'), ('masing', 'OTHER'), ('-', 'X'), ('masing', 'X'), ('Dalam', 'OTHER'), ('video', 'OTHER'), ('pendek', 'OTHER'), ('terbitan', 'organization'), ('Jabatan', 'organization'), ('Keselamatan', 'organization'), ('Jalan', 'organization'), ('Raya', 'event'), ('(', 'organization'), ('Jkjr', 'X'), (')', 'X'), ('itu', 'OTHE

In [38]:
file = 'fastformer-base-entities/frozen_model.pb'
outPutname = 'entity/fastformer/model.pb'
b2_bucket.upload_local_file(
    local_file=file,
    file_name=outPutname,
    file_infos=file_info,
)

In [39]:
file = 'fastformer-base-entities/frozen_model.pb.quantized'
outPutname = 'entity/fastformer-quantized/model.pb'
b2_bucket.upload_local_file(
    local_file=file,
    file_name=outPutname,
    file_infos=file_info,
)